In [1]:
from __future__ import print_function

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [2]:
# SparkSession 생성 (참고: config 섹션은 Windows 용입니다!)
spark = SparkSession.builder.appName("DecisionTree").getOrCreate()


In [3]:
# 데이터를 데이터프레임으로 로드합니다.
# realestate.csv 파일을 읽어들여 데이터프레임으로 저장
data = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("file:///SparkCourse/realestate.csv")  
print(data)
data.collect()

DataFrame[No: int, TransactionDate: double, HouseAge: double, DistanceToMRT: double, NumberConvenienceStores: int, Latitude: double, Longitude: double, PriceOfUnitArea: double]


[Row(No=1, TransactionDate=2012.917, HouseAge=32.0, DistanceToMRT=84.87882, NumberConvenienceStores=10, Latitude=24.98298, Longitude=121.54024, PriceOfUnitArea=37.9),
 Row(No=2, TransactionDate=2012.917, HouseAge=19.5, DistanceToMRT=306.5947, NumberConvenienceStores=9, Latitude=24.98034, Longitude=121.53951, PriceOfUnitArea=42.2),
 Row(No=3, TransactionDate=2013.583, HouseAge=13.3, DistanceToMRT=561.9845, NumberConvenienceStores=5, Latitude=24.98746, Longitude=121.54391, PriceOfUnitArea=47.3),
 Row(No=4, TransactionDate=2013.5, HouseAge=13.3, DistanceToMRT=561.9845, NumberConvenienceStores=5, Latitude=24.98746, Longitude=121.54391, PriceOfUnitArea=54.8),
 Row(No=5, TransactionDate=2012.833, HouseAge=5.0, DistanceToMRT=390.5684, NumberConvenienceStores=5, Latitude=24.97937, Longitude=121.54245, PriceOfUnitArea=43.1),
 Row(No=6, TransactionDate=2012.667, HouseAge=7.1, DistanceToMRT=2175.03, NumberConvenienceStores=3, Latitude=24.96305, Longitude=121.51254, PriceOfUnitArea=32.1),
 Row(No=

In [5]:
assembler = VectorAssembler().setInputCols(["HouseAge", "DistanceToMRT", \
                            "NumberConvenienceStores"]).setOutputCol("features")
assembler  # VectorAssembler 객체 생성 및 변수에 할당

# setInputCols 메서드를 사용하여 입력 컬럼을 설정
# "HouseAge", "DistanceToMRT", "NumberConvenienceStores" 컬럼을 입력으로 사용

# setOutputCol 메서드를 사용하여 출력 컬럼을 설정
# "features" 컬럼을 출력으로 사용

VectorAssembler_3e75ab39d8f9

In [6]:
# 데이터를 변환하고 "PriceOfUnitArea"와 "features" 열만 선택하여 df에 할당
df = assembler.transform(data).select("PriceOfUnitArea", "features")  
print(df)
df.collect()  # df를 수집하여 결과를 반환

DataFrame[PriceOfUnitArea: double, features: vector]


[Row(PriceOfUnitArea=37.9, features=DenseVector([32.0, 84.8788, 10.0])),
 Row(PriceOfUnitArea=42.2, features=DenseVector([19.5, 306.5947, 9.0])),
 Row(PriceOfUnitArea=47.3, features=DenseVector([13.3, 561.9845, 5.0])),
 Row(PriceOfUnitArea=54.8, features=DenseVector([13.3, 561.9845, 5.0])),
 Row(PriceOfUnitArea=43.1, features=DenseVector([5.0, 390.5684, 5.0])),
 Row(PriceOfUnitArea=32.1, features=DenseVector([7.1, 2175.03, 3.0])),
 Row(PriceOfUnitArea=40.3, features=DenseVector([34.5, 623.4731, 7.0])),
 Row(PriceOfUnitArea=46.7, features=DenseVector([20.3, 287.6025, 6.0])),
 Row(PriceOfUnitArea=18.8, features=DenseVector([31.7, 5512.038, 1.0])),
 Row(PriceOfUnitArea=22.1, features=DenseVector([17.9, 1783.18, 3.0])),
 Row(PriceOfUnitArea=41.4, features=DenseVector([34.8, 405.2134, 1.0])),
 Row(PriceOfUnitArea=58.1, features=DenseVector([6.3, 90.4561, 9.0])),
 Row(PriceOfUnitArea=39.3, features=DenseVector([13.0, 492.2313, 5.0])),
 Row(PriceOfUnitArea=23.8, features=DenseVector([20.4, 24

In [7]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할합니다.
trainTest = df.randomSplit([0.5, 0.5])  # 데이터를 50:50 비율로 무작위로 분할합니다.
trainingDF = trainTest[0]  # 훈련 데이터로 사용할 데이터 프레임
testDF = trainTest[1]  # 테스트 데이터로 사용할 데이터 프레임

# 데이터 프레임의 전체 길이를 출력합니다.
print(len(df.collect()))

# 훈련 데이터 프레임의 길이를 출력합니다.
print(len(trainingDF.collect()))

# 테스트 데이터 프레임의 길이를 출력합니다.
print(len(testDF.collect()))

414
206
208


In [8]:

# 이제 우리의 의사 결정 트리를 생성합니다.
dtr = DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("PriceOfUnitArea")
dtr

DecisionTreeRegressor_d310983a147a

In [9]:
# Train the model using our training data
model = dtr.fit(trainingDF)  # 모델을 학습 데이터로 훈련시킵니다.
model  # 모델 출력

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_d310983a147a, depth=5, numNodes=57, numFeatures=3

In [11]:
# 이제 테스트 데이터에서 값을 예측해보겠습니다.
# 테스트 데이터프레임의 모든 특성에 대해 결정 트리 모델을 사용하여 예측을 생성합니다:
fullPredictions = model.transform(testDF).cache()
print(fullPredictions)
fullPredictions.collect()

DataFrame[PriceOfUnitArea: double, features: vector, prediction: double]


[Row(PriceOfUnitArea=11.2, features=DenseVector([17.4, 6488.021, 1.0]), prediction=16.200000000000003),
 Row(PriceOfUnitArea=11.6, features=DenseVector([16.0, 4066.587, 0.0]), prediction=29.299999999999997),
 Row(PriceOfUnitArea=12.2, features=DenseVector([30.9, 6396.283, 1.0]), prediction=24.699999999999996),
 Row(PriceOfUnitArea=12.8, features=DenseVector([32.0, 1156.777, 0.0]), prediction=23.542105263157893),
 Row(PriceOfUnitArea=13.0, features=DenseVector([13.6, 4197.349, 0.0]), prediction=16.833333333333332),
 Row(PriceOfUnitArea=13.2, features=DenseVector([29.4, 4510.359, 1.0]), prediction=22.6),
 Row(PriceOfUnitArea=13.7, features=DenseVector([31.7, 1160.632, 0.0]), prediction=23.542105263157893),
 Row(PriceOfUnitArea=13.8, features=DenseVector([31.7, 1159.454, 0.0]), prediction=23.542105263157893),
 Row(PriceOfUnitArea=14.4, features=DenseVector([24.0, 4527.687, 0.0]), prediction=17.3),
 Row(PriceOfUnitArea=15.6, features=DenseVector([13.8, 4082.015, 0.0]), prediction=16.833333

In [14]:
# 예측값과 "알려진" 정확한 레이블을 추출합니다.
predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])  # 예측값을 추출하여 RDD로 변환합니다.
labels = fullPredictions.select("PriceOfUnitArea").rdd.map(lambda x: x[0])  # "PriceOfUnitArea" 레이블을 추출하여 RDD로 변환합니다.
display(predictions)
display(predictions.collect())
print('='*80)
print('='*80)
print('='*80)
display(labels)
display(labels.collect())

PythonRDD[107] at RDD at PythonRDD.scala:53

[16.200000000000003,
 29.299999999999997,
 24.699999999999996,
 23.542105263157893,
 16.833333333333332,
 22.6,
 23.542105263157893,
 23.542105263157893,
 17.3,
 16.833333333333332,
 17.3,
 24.699999999999996,
 23.542105263157893,
 23.542105263157893,
 16.833333333333332,
 24.699999999999996,
 23.542105263157893,
 16.833333333333332,
 16.833333333333332,
 23.542105263157893,
 23.542105263157893,
 28.117391304347827,
 23.542105263157893,
 28.117391304347827,
 23.542105263157893,
 29.299999999999997,
 23.542105263157893,
 23.542105263157893,
 23.542105263157893,
 34.49333333333333,
 23.542105263157893,
 29.9,
 23.542105263157893,
 28.117391304347827,
 28.117391304347827,
 23.542105263157893,
 23.542105263157893,
 23.542105263157893,
 34.49333333333333,
 28.117391304347827,
 40.158620689655166,
 24.4,
 23.542105263157893,
 28.117391304347827,
 34.49333333333333,
 28.117391304347827,
 49.666666666666664,
 28.117391304347827,
 23.520000000000003,
 16.833333333333332,
 23.542105263157893,
 2

PythonRDD[108] at RDD at PythonRDD.scala:53

[11.2,
 11.6,
 12.2,
 12.8,
 13.0,
 13.2,
 13.7,
 13.8,
 14.4,
 15.6,
 15.6,
 17.4,
 18.3,
 18.3,
 18.6,
 18.8,
 19.0,
 19.2,
 20.0,
 20.7,
 20.7,
 20.9,
 20.9,
 21.3,
 21.5,
 21.8,
 21.8,
 22.1,
 22.8,
 23.0,
 23.1,
 23.2,
 23.5,
 23.6,
 23.6,
 23.7,
 23.8,
 23.8,
 24.5,
 24.8,
 25.0,
 25.6,
 25.7,
 26.2,
 26.5,
 26.6,
 26.9,
 27.0,
 27.3,
 27.3,
 27.7,
 27.7,
 28.4,
 28.4,
 28.6,
 28.8,
 29.3,
 29.4,
 29.5,
 29.5,
 29.8,
 30.1,
 30.6,
 30.6,
 30.7,
 31.1,
 31.1,
 31.3,
 31.3,
 31.6,
 31.7,
 31.9,
 32.4,
 32.5,
 32.9,
 32.9,
 33.1,
 33.1,
 34.0,
 34.1,
 34.1,
 34.2,
 34.6,
 34.6,
 34.7,
 35.1,
 35.5,
 35.6,
 35.7,
 36.2,
 36.3,
 36.3,
 36.5,
 36.5,
 36.6,
 36.7,
 36.7,
 36.8,
 36.9,
 37.0,
 37.4,
 37.4,
 37.4,
 37.5,
 37.5,
 37.7,
 37.8,
 37.9,
 38.4,
 39.3,
 39.4,
 39.6,
 39.7,
 39.7,
 40.2,
 40.5,
 40.6,
 40.6,
 40.6,
 40.8,
 40.8,
 40.9,
 41.0,
 41.0,
 41.1,
 41.2,
 41.2,
 41.4,
 41.6,
 42.0,
 42.0,
 42.0,
 42.1,
 42.2,
 42.2,
 42.2,
 42.3,
 42.5,
 42.6,
 42.8,
 43.1,
 43.1,
 43.2,

In [12]:
predictions.zip(labels)

org.apache.spark.api.java.JavaPairRDD@1d22d1fc

In [13]:

# Zip them together
predictionAndLabel = predictions.zip(labels).collect()
predictionAndLabel

[(39.54509803921569, 7.6),
 (17.23333333333333, 12.2),
 (13.925, 12.8),
 (16.225, 12.8),
 (25.38695652173913, 12.9),
 (17.3, 13.0),
 (17.23333333333333, 13.2),
 (13.925, 14.7),
 (13.925, 15.0),
 (22.166666666666668, 15.6),
 (13.925, 15.6),
 (17.3, 15.9),
 (25.5, 16.1),
 (17.23333333333333, 17.4),
 (25.38695652173913, 18.2),
 (17.3, 18.6),
 (13.925, 18.8),
 (16.225, 19.0),
 (24.22, 20.7),
 (25.5, 20.8),
 (25.38695652173913, 20.9),
 (25.38695652173913, 20.9),
 (31.0, 21.4),
 (24.22, 21.5),
 (32.68333333333334, 21.7),
 (25.38695652173913, 21.8),
 (25.38695652173913, 21.8),
 (17.23333333333333, 22.1),
 (22.0, 22.3),
 (25.5, 22.3),
 (17.23333333333333, 22.6),
 (25.38695652173913, 22.8),
 (39.54509803921569, 22.8),
 (33.916666666666664, 23.0),
 (25.96666666666667, 23.1),
 (25.5, 23.1),
 (25.38695652173913, 23.6),
 (16.225, 23.8),
 (39.54509803921569, 24.5),
 (28.18, 24.6),
 (22.0, 24.7),
 (46.60000000000001, 24.7),
 (17.23333333333333, 24.7),
 (25.38695652173913, 24.8),
 (39.54509803921569, 

In [14]:

# Print out the predicted and actual values for each point
for prediction in predictionAndLabel:
    print(prediction)


# Stop the session
spark.stop()

(39.54509803921569, 7.6)
(17.23333333333333, 12.2)
(13.925, 12.8)
(16.225, 12.8)
(25.38695652173913, 12.9)
(17.3, 13.0)
(17.23333333333333, 13.2)
(13.925, 14.7)
(13.925, 15.0)
(22.166666666666668, 15.6)
(13.925, 15.6)
(17.3, 15.9)
(25.5, 16.1)
(17.23333333333333, 17.4)
(25.38695652173913, 18.2)
(17.3, 18.6)
(13.925, 18.8)
(16.225, 19.0)
(24.22, 20.7)
(25.5, 20.8)
(25.38695652173913, 20.9)
(25.38695652173913, 20.9)
(31.0, 21.4)
(24.22, 21.5)
(32.68333333333334, 21.7)
(25.38695652173913, 21.8)
(25.38695652173913, 21.8)
(17.23333333333333, 22.1)
(22.0, 22.3)
(25.5, 22.3)
(17.23333333333333, 22.6)
(25.38695652173913, 22.8)
(39.54509803921569, 22.8)
(33.916666666666664, 23.0)
(25.96666666666667, 23.1)
(25.5, 23.1)
(25.38695652173913, 23.6)
(16.225, 23.8)
(39.54509803921569, 24.5)
(28.18, 24.6)
(22.0, 24.7)
(46.60000000000001, 24.7)
(17.23333333333333, 24.7)
(25.38695652173913, 24.8)
(39.54509803921569, 25.0)
(31.0, 25.3)
(25.5, 25.3)
(24.22, 25.3)
(32.68333333333334, 25.6)
(25.3869565217391